# Converting Pytorch model to OpenVino

In [12]:
import io
import os
import hashlib
import tempfile
import requests
import numpy as np
from PIL import Image
from argparse import ArgumentParser

import torch
import torchvision.transforms as transforms

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

# for example we are using the Resnet18
from torchvision.models.resnet import resnet18 as net

In [13]:
# define helper functions
def fetch(url):
  # efficient loading of URLS
  fp = os.path.join(tempfile.gettempdir(), hashlib.md5(url.encode('utf-8')).hexdigest())
  if os.path.isfile(fp) and os.stat(fp).st_size > 0:
    with open(fp, "rb") as f:
      dat = f.read()
  else:
    print("fetching", url)
    dat = requests.get(url).content
    with open(fp+".tmp", "wb") as f:
      f.write(dat)
    os.rename(fp+".tmp", fp)
  return dat

def get_image_net_labels():
  with open("labels.txt", 'r') as f:
    labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip()[10:] for x in f]
  return labels_map

In [14]:
labels = get_image_net_labels()
# load the model by downloading pretrained weights
model = net(pretrained=True)
_ = model.eval()

In [15]:
# define the image transformations to apply to each image
image_transformations = transforms.Compose([
    transforms.Resize(256),                               # resize to a (256,256) image
    transforms.CenterCrop(224),                           # crop centre part of image to get (244, 244) grid
    transforms.ToTensor(),                                # convert to tensor
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),    # normalise image according to imagenet valuess
])

In [17]:
# add URL below for simple run
url = "some/url/to/some/image.jpg"
image = Image.open(io.BytesIO(fetch(url)))  # load any image
x = image_transformations(image) # [3, H, W]
x = x.view(1, *x.size())
x.size()

torch.Size([1, 3, 224, 224])

In [18]:
with torch.no_grad():
    out = model(x)

In [19]:
# keep the top 10 scores
number_top = 10
probs, indices = torch.topk(out, number_top)
print(f"Top {number_top} results:")
print("===============")
for p, i in zip(probs[0], indices[0]):
    print(p, "--", labels[i])

Top 10 results:
tensor(27.3914) -- cheeseburger
tensor(17.5533) -- bagel,_beigel
tensor(15.8071) -- hotdog,_hot_dog,_red_hot
tensor(15.2610) -- guacamole
tensor(13.3716) -- French_loaf
tensor(12.9023) -- mushroom
tensor(12.6587) -- meat_loaf,_meatloaf
tensor(12.4445) -- bolete
tensor(12.3000) -- broccoli
tensor(11.9797) -- bakery,_bakeshop,_bakehouse


In [21]:
# convert to ONNX
print("-"*70)
print("Now we are converting this pytorch model to ONNX model")
torch.onnx._export(model, x, f'resnet18.onnx', export_params=True)
print(f"See in the folder we have a resnet18.onnx file")
print("-"*70)

----------------------------------------------------------------------
Now we are converting this pytorch model to ONNX model
See in the folder we have a resnet18.onnx file
----------------------------------------------------------------------


### ONNX to OpenVino

For this piece please consult the README since it has CLI.

In [ ]:
from openvino.inference_engine import IECore

In [24]:
model_xml = "fp32/resnet18.xml"
model_bin = os.path.splitext(model_xml)[0] + ".bin"
model_xml, model_bin

('fp32/resnet18.xml', 'fp32/resnet18.bin')

In [ ]:
# Plugin initialization for specified device and load extensions library if specified.
print("Creating Inference Engine...")
ie = IECore()

# Read IR
print("Loading network")
net = ie.read_network(args.model, os.path.splitext(args.model)[0] + ".bin")

print("Loading IR to the plugin...")
exec_net = ie.load_network(network=net, device_name="CPU", num_requests=2)
print(f"exec_net: {exec_net}")
print("-"*70)

In [ ]:
# convert to numpy to pass this to the IECore
x = x.numpy()

In [ ]:
# this is a bit tricky. So the input to the model is the input from ONNX graph
# IECore makes a networkX graph of the "computation graph" and when we run .infer
# it passes it through. If you are unsure of what to pass you can always check the
# <model>.xml file. In case of pytorch models the value "input.1" is the usual
# suspect. Happy Hunting!
out = exec_net.infer(inputs={"input.1": x})

# the output looks like this {"node_id": array()} so we simply load the output
out = list(out.values())[0]
print("Output Shape:", out.shape)
print("-"*70)

In [ ]:
# keep the top 10 scores
out = out[0]
number_top = 10
indices = np.argsort(out, -1)[::-1][:number_top]
probs = out[indices]
print(f"Top {number_top} results:")
print("===============")
for p, i in zip(probs, indices):
print(p, "--", labels[i])
print("-"*70)

## Conclusion

Using this method you can convert any arbitrary pytorch model to OpenVino!